In [104]:
%reset -f

In [105]:
%who

Interactive namespace is empty.


### Подключение и извлечение данных из БД

In [106]:
import psycopg2
import pandas as pd
conf = {
    'dbname': 'demo',
    'user': 'user',
    'password': 'user12345678',
    'host': 'localhost'
}

In [107]:
class DataBase(object):
    def __init__(self, params: dict):
        self.params = params
        self.conn = None

    def connection_db(self):
        connect = psycopg2.connect(**self.params)
        self.conn = connect

    def close_connection_db(self):
        self.conn.close()


class WorkDataBase(object):
    def __init__(self, params):
        self.params = params
        self.database = DataBase(params)
        self.cursor = None
        self.tables = []

    def create_cursor_database(self):
        self.database.connection_db()
        self.cursor = self.database.conn.cursor()

    def get_tables(self):
        self.create_cursor_database()
        self.cursor.execute("""
            SELECT table_name FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name ASC
        """)
        self.tables = [table[0] for table in self.cursor.fetchall()]
        self.database.close_connection_db()

    def get_info_tables(self):
        self.create_cursor_database()
        summary = {
            'empty': [],
            'no_empty': {}
        }

        for table in self.tables:
            self.cursor.execute(f"SELECT * FROM {table} LIMIT 50")
            struct_table = self.cursor.description
            data = self.cursor.fetchall()
            if not data:
                summary['empty'].append(table)
                continue

            summary['no_empty'].setdefault(table, dict())
            for row in data:
                for n in range(len(row)):
                    summary['no_empty'][table].setdefault(struct_table[n].name, [])
                    summary['no_empty'][table][struct_table[n].name].append(str(row[n]))
        self.database.close_connection_db()
        
        return summary

In [108]:
connect_db1 = WorkDataBase(conf)
connect_db1.get_tables()
all_tables = connect_db1.tables
summary = connect_db1.get_info_tables()

### Подготовка и экспорт данных в Excel

In [ ]:
len(all_tables)

In [115]:
no_empty = [key for key in summary['no_empty'].keys()]
len(no_empty)

In [111]:
def check_empty(table):
    if table in no_empty:
        return "NO"
    return "YES"

df_all_tables = pd.DataFrame({
    'NAME_TABLES': all_tables,
})

df_all_tables['EMPTY'] = df_all_tables['NAME_TABLES'].apply(check_empty)

In [112]:
with pd.ExcelWriter('data.xlsx') as docum:
    df_all_tables.to_excel(docum, sheet_name='SUMMARY', index=False)

In [113]:
with pd.ExcelWriter('data.xlsx', mode='a') as docum:
    for table in no_empty:
        df_table = pd.DataFrame(summary['no_empty'][table])
        df_table.to_excel(docum, sheet_name=table, index=False)
        

In [ ]:
len(pd.ExcelFile('data.xlsx').sheet_names)